In [28]:
pip install --user -U nltk

Note: you may need to restart the kernel to use updated packages.


ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.


In [29]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


Importing necessary libraries

In [30]:
import pandas as pd
import numpy as np
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\neels\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\neels\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\neels\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Reading the csv file

In [31]:
df=pd.read_csv('data.csv',encoding='latin1')

In [32]:
print(df.head())
print(df.info())

   Unnamed: 0               0  \
0           0  Thyroid_Cancer   
1           1  Thyroid_Cancer   
2           2  Thyroid_Cancer   
3           3  Thyroid_Cancer   
4           4  Thyroid_Cancer   

                                                   a  
0  Thyroid surgery in  children in a single insti...  
1  " The adopted strategy was the same as that us...  
2  coronary arterybypass grafting thrombosis ï¬b...  
3   Solitary plasmacytoma SP of the skull is an u...  
4   This study aimed to investigate serum matrix ...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7570 entries, 0 to 7569
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  7570 non-null   int64 
 1   0           7570 non-null   object
 2   a           7570 non-null   object
dtypes: int64(1), object(2)
memory usage: 177.6+ KB
None


Dropping unnecessary columns 

In [33]:
columns_to_drop = ['Unnamed: 0'] 
df = df.drop(columns=columns_to_drop)

Checking for missing values

In [34]:
missing_values = df.isnull().sum()
print(missing_values)

0    0
a    0
dtype: int64


Data Preprocessing 

In [35]:
    def preprocess_text(df, text_column):
        # Normalize case
        df[text_column] = df[text_column].str.lower()
        
        # Remove punctuation
        df[text_column] = df[text_column].str.translate(str.maketrans('', '', string.punctuation))
        
        # Tokenization
        df['tokens'] = df[text_column].apply(word_tokenize)
        
        # Remove stop words
        stop_words = set(stopwords.words('english'))
        df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])
        
        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        df['tokens'] = df['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
        
        # Join tokens back to string
        df['cleaned_text'] = df['tokens'].apply(lambda x: ' '.join(x))
        
        return df
    
    # Preprocess the data
    df = preprocess_text(df, 'a')

In [36]:
from transformers import AutoTokenizer, AutoModel
import torch

def extract_biobert_embeddings(texts):
    tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.1')
    model = AutoModel.from_pretrained('dmis-lab/biobert-base-cased-v1.1')

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    print(f"Using device: {device}")

    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
        inputs = {key: val.to(device) for key, val in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy())
    return np.array(embeddings)

texts = df['cleaned_text'].tolist()
X_biobert = extract_biobert_embeddings(texts)
print(X_biobert)


Using device: cuda


D:\Tamanna Python\myenv\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


[[ 0.04956692 -0.33872783 -0.30338883 ...  0.14580292 -0.26159668
   0.13023537]
 [ 0.1531326  -0.4530626  -0.30261266 ... -0.07585327 -0.19635022
   0.2660275 ]
 [ 0.15856275 -0.20920467 -0.24372411 ... -0.01992516 -0.13394508
   0.0904753 ]
 ...
 [ 0.1127191  -0.1103319  -0.30815363 ... -0.07805791 -0.10341207
   0.04934892]
 [ 0.14343181 -0.14478391 -0.10110567 ...  0.00631718 -0.08262135
  -0.02507709]
 [ 0.07725459 -0.38750505 -0.27121773 ... -0.0347251  -0.27841103
   0.12252714]]


In [37]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['labels'] = label_encoder.fit_transform(df['0']) 

In [38]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_biobert, df['labels'], test_size=0.2, random_state=42)

In [39]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)

model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [40]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")
print(f"Confusion Matrix:\n{conf_matrix}")


Accuracy: 0.9286657859973579
Classification Report:
                precision    recall  f1-score   support

  Colon_Cancer       0.86      0.94      0.90       517
   Lung_Cancer       1.00      1.00      1.00       407
Thyroid_Cancer       0.94      0.87      0.90       590

      accuracy                           0.93      1514
     macro avg       0.94      0.94      0.93      1514
  weighted avg       0.93      0.93      0.93      1514

Confusion Matrix:
[[487   0  30]
 [  0 407   0]
 [ 78   0 512]]


In [41]:
from sklearn.svm import SVC

model = SVC(kernel='linear', C=1)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")
print(f"Confusion Matrix:\n{conf_matrix}")


Accuracy: 0.9359313077939234
Classification Report:
                precision    recall  f1-score   support

  Colon_Cancer       0.87      0.95      0.91       517
   Lung_Cancer       1.00      1.00      1.00       407
Thyroid_Cancer       0.96      0.88      0.91       590

      accuracy                           0.94      1514
     macro avg       0.94      0.94      0.94      1514
  weighted avg       0.94      0.94      0.94      1514

Confusion Matrix:
[[493   0  24]
 [  0 407   0]
 [ 73   0 517]]


In [42]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")
print(f"Confusion Matrix:\n{conf_matrix}")


Accuracy: 0.9894319682959049
Classification Report:
                precision    recall  f1-score   support

  Colon_Cancer       0.98      0.99      0.98       517
   Lung_Cancer       1.00      1.00      1.00       407
Thyroid_Cancer       0.99      0.98      0.99       590

      accuracy                           0.99      1514
     macro avg       0.99      0.99      0.99      1514
  weighted avg       0.99      0.99      0.99      1514

Confusion Matrix:
[[513   0   4]
 [  0 407   0]
 [ 12   0 578]]


In [43]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")


Accuracy: 0.987450462351387


In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(512, input_dim=X_biobert.shape[1], activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Neural Network Accuracy: {accuracy}")

Epoch 1/10


D:\Tamanna Python\myenv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5995 - loss: 0.7231 - val_accuracy: 0.7888 - val_loss: 0.4317
Epoch 2/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7788 - loss: 0.4218 - val_accuracy: 0.7690 - val_loss: 0.4194
Epoch 3/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7922 - loss: 0.4124 - val_accuracy: 0.7979 - val_loss: 0.3828
Epoch 4/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8394 - loss: 0.3239 - val_accuracy: 0.8639 - val_loss: 0.2859
Epoch 5/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8717 - loss: 0.2812 - val_accuracy: 0.8936 - val_loss: 0.2291
Epoch 6/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8886 - loss: 0.2398 - val_accuracy: 0.8960 - val_loss: 0.2079
Epoch 7/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9116 - loss: 0.1889 - val_accuracy: 0.8985 - val_loss: 0.2133
Epoch 8/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9075 - loss: 0.1967 - val_accuracy: 0.8911 - val_

In [45]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [49]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict

dataset = Dataset.from_pandas(df)
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.1')

def tokenize_function(examples):
    # Tokenize the text
    model_inputs = tokenizer(
        examples['cleaned_text'],
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors="pt"
    )
    model_inputs['labels'] = examples['labels']
    return model_inputs


tokenized_dataset = dataset.map(tokenize_function, batched=True)


train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
datasets = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})


num_labels = 3 
model = BertForSequenceClassification.from_pretrained(
    'dmis-lab/biobert-base-cased-v1.1',
    num_labels=num_labels
)

for param in model.parameters():
    param.data = param.data.contiguous()

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    fp16=torch.cuda.is_available()  # Enable mixed precision if CUDA is available
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=datasets['train'],
    eval_dataset=datasets['test']
)


trainer.train()


results = trainer.evaluate()
print("Results from BERT Model Training:", results)


Map: 100%|██████████████████████████████████████████████████████████████████| 7570/7570 [03:06<00:00, 40.70 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.504900,0.087028
2,0.064800,0.055279
3,0.033000,0.028791


Results from BERT Model Training: {'eval_loss': 0.028791379183530807, 'eval_runtime': 12.6675, 'eval_samples_per_second': 119.519, 'eval_steps_per_second': 14.999, 'epoch': 3.0}


In [50]:
from sklearn.metrics import classification_report, confusion_matrix

predictions = trainer.predict(test_dataset=datasets['test'])
pred_labels = predictions.predictions.argmax(-1)

print(classification_report(datasets['test']['labels'], pred_labels))

conf_mat = confusion_matrix(datasets['test']['labels'], pred_labels)
print("Confusion Matrix:\n", conf_mat)


              precision    recall  f1-score   support

           0       0.99      0.98      0.99       515
           1       1.00      1.00      1.00       425
           2       0.98      0.99      0.99       574

    accuracy                           0.99      1514
   macro avg       0.99      0.99      0.99      1514
weighted avg       0.99      0.99      0.99      1514

Confusion Matrix:
 [[505   0  10]
 [  0 425   0]
 [  4   0 570]]


In [51]:
model.save_pretrained('Model')
tokenizer.save_pretrained('Tokenizer')

('Tokenizer\\tokenizer_config.json',
 'Tokenizer\\special_tokens_map.json',
 'Tokenizer\\vocab.txt',
 'Tokenizer\\added_tokens.json')